<h2>Problem Statement # 1 :  Getting ready to appear for a Trial</h2>

In [ ]:
!pip install PyPDF2
!pip install openai
!pip install nltk
!pip install python-dotenv

In [6]:
import platform
import os
import openai
import os
import re
import PyPDF2

In [7]:
#load api key from .env
from dotenv import load_dotenv
load_dotenv()
openai.api_key=os.getenv("OPENAI_API_KEY")

In [8]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

print('Python: ', platform.python_version())
print('re: ', re.__version__)
print('nltk: ', nltk.__version__)

Python:  3.9.0
re:  2.2.1
nltk:  3.8.1


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lnavn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
# Read the PDF document
def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text

In [10]:
def count_tokens(text):
    tokens = word_tokenize(text)
    return len(tokens)

In [ ]:
text = read_pdf("ps1.pdf")
token_count = count_tokens(text)
print(f"Number of tokens: {token_count}")

In [ ]:
def break_up_file(tokens, chunk_size, overlap_size):
    if len(tokens) <= chunk_size:
        yield tokens
    else:
        chunk = tokens[:chunk_size]
        yield chunk
        yield from break_up_file(tokens[chunk_size-overlap_size:], chunk_size, overlap_size)

def break_up_file_to_chunks(text, chunk_size=2000, overlap_size=100):
    tokens = word_tokenize(text)
    return list(break_up_file(tokens, chunk_size, overlap_size))

In [ ]:
chunks = break_up_file_to_chunks(text)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {len(chunk)} tokens")

In [ ]:
def convert_to_detokenized_text(tokenized_text):
    prompt_text = " ".join(tokenized_text)
    prompt_text = prompt_text.replace(" 's", "'s")
    return prompt_text

In [ ]:
prompt_response = []
chunks = break_up_file_to_chunks(text)

In [ ]:
for i, chunk in enumerate(chunks):
    prompt_request = "Please generate a concise summary for the following text:\n" + convert_to_detokenized_text(chunks[i])
    response = openai.Completion.create(
            model="text-davinci-003",
            prompt=prompt_request,
            temperature=.5,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
    )
    
    prompt_response.append(response["choices"][0]["text"].strip())

In [ ]:
prompt_request = "Give the 10 most important items pertaining this trial: " + str(prompt_response)

In [ ]:
response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt_request,
        temperature=.5,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

meeting_summary = response["choices"][0]["text"].strip()

# Split the meeting summary into sentences
sentences = sent_tokenize(meeting_summary)

# Print the sentences with bullet points
for i, sentence in enumerate(sentences, start=1):
    sentence = sentence.strip()
    if sentence != "":
        print(f"{sentence}")
